# Part 1: Linear Regression Tutorial with Ames Housing Dataset  

This notebook will guide you step by step through **linear regression**, from simple to more complex models, using the Ames Housing dataset.  

Learning Objectives:  
- Understand the basics of **linear regression** for predictive modeling.  
- Implement **simple linear regression** with one feature and interpret coefficients.  
- Extend to **multiple linear regression** with several features and evaluate performance.  
- Explore **polynomial regression** to capture nonlinear relationships.  
- Recognize the problem of **overfitting** when using higher-degree polynomials.  
- Apply **Ridge regularization** to control overfitting and balance the bias–variance tradeoff.  
- Evaluate models using **RMSE**, **R²**, and **Predicted vs Observed plots**.  


## 1.1 Load Dataset  


💡 **Tips**  

- **fetch_openml()**  
  - Function from scikit-learn to download datasets from the [OpenML repository](https://www.openml.org/).  
  - Setting `as_frame=True` returns the dataset as a **pandas DataFrame**, making it easier to explore with pandas tools.  
  
- **Ames Housing Dataset**  
  - Contains **1460 houses × 81 columns**.  
  - **Features (80 columns):** lot size, number of rooms, year built, etc.  
  - **Target (1 column):** `SalePrice`, the house price we want to predict.  
  - Often used as a benchmark in regression tutorials. 

- **Suppress warnings**  
  - `warnings.filterwarnings()` hides solver/runtime warnings that may appear (especially with high-degree polynomials), keeping the notebook output clean.  

- **First look at the data**  
  - `print("Shape:", df.shape)` shows dataset dimensions.  
  - `df.head()` displays the first 5 rows for inspection.  



In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

housing = fetch_openml(name="house_prices", as_frame=True)
df = housing.frame

print("Shape:", df.shape)
df.head()

## 1.2 Select and Visualize Features  

💡 **Tips:**   
- **Feature selection**: 
- Use `pandas` to select relevant numeric columns. 
- We pick a few numeric columns that are likely related to house price (check the [explanation](https://www.kaggle.com/competitions/regression-techniques-house-prices/data) of each feature):
  - `GrLivArea` (living area in sqft)  
  - `BedroomAbvGr` (bedrooms above ground)  
  - `GarageCars` (garage capacity)  
  - `FullBath` (full bathrooms)  
  - `LotArea` (lot size in sqft)  
  - `YearBuilt` (year the house was built)  
  - `TotRmsAbvGrd` (total rooms above ground)  
  - `Fireplaces` (number of fireplaces)  
- **Target variable**:  
  - We use `SalePrice`, scaled down by 1000 for readability (so prices are in thousands).  
- **Scatter plots:**  
    - `plt.scatter()` shows how each feature relates to the target.  
    - A stronger linear trend (e.g., **GrLivArea vs SalePrice**) suggests that **linear regression** may fit well. 
- **Subplots setup:**  
    - `plt.subplot(131)` creates the **first** plot in a row of 3.  
    - `132`, `133` specify the **second** and **third** plots.  
- **Other details:**  
    - `alpha=0.6` makes points **semi-transparent**, so overlaps are easier to see.  
    - `plt.tight_layout()` prevents titles/labels from overlapping.  

In [ ]:
# Select numeric features 
X = df[["GrLivArea", "BedroomAbvGr", "GarageCars"]]
y = df["SalePrice"] / 1000  # in thousands for readability

# Scatter plots using subplot(131), (132), (133)
plt.figure(figsize=(12, 4))

plt.subplot(131)
plt.scatter(X["GrLivArea"], y, alpha=0.6)
plt.title("GrLivArea vs SalePrice")
plt.xlabel("GrLivArea")
plt.ylabel("SalePrice ($1000s)")

plt.subplot(132)
plt.scatter(X["BedroomAbvGr"], y, alpha=0.6)
plt.title("BedroomAbvGr vs SalePrice")
plt.xlabel("BedroomAbvGr")
plt.ylabel("SalePrice ($1000s)")

plt.subplot(133)
plt.scatter(X["GarageCars"], y, alpha=0.6)
plt.title("GarageCars vs SalePrice")
plt.xlabel("GarageCars")
plt.ylabel("SalePrice ($1000s)")

plt.tight_layout()
plt.show()

## 1.3 Simple Linear Regression  

💡 **Tips**  

- **Data splitting**  
  - `train_test_split()` divides the dataset into training and test sets.  
  - Training set: used to fit (learn) the model.  
  - Test set: used to evaluate how well the model generalizes to unseen data.  

- **Model training**  
  - `LinearRegression().fit()` finds the best-fit line by minimizing the sum of squared errors.  
  - `model.coef_` = slope of the line (effect of the feature on price).  
  - `model.intercept_` = intercept (baseline price when the feature = 0).  

- **Evaluation metrics**  
  - `mean_squared_error()` → measures prediction error (lower is better).  
  - `r2_score()` → measures how much variance in the target is explained by the model (closer to 1 is better).  

- **Train vs Test comparison**  
  - Training performance shows how well the model fits the data it learned from.  
  - Test performance shows how well it generalizes to new data.  
  - Always compare both to detect **underfitting** (poor train + test) or **overfitting** (good train, poor test).  

- **Visualization**  
  - Scatter plot: predicted vs. observed sale prices on the test set.  
  - The red dashed line represents a perfect prediction line (`y = x`).  
  - The closer the points are to this line, the better the model performance.  


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

X1 = X[["GrLivArea"]]  # single feature
X1_train, X1_test, y_train, y_test = train_test_split(X1, y, test_size=0.3, random_state=42)

model1 = LinearRegression()
model1.fit(X1_train, y_train)

y_train_pred = model1.predict(X1_train)
y_test_pred = model1.predict(X1_test)

print("Coefficient:", model1.coef_[0])
print("Intercept:", model1.intercept_)
print("Train RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))
print("Train R²:", r2_score(y_train, y_train_pred))
print("Test R²:", r2_score(y_test, y_test_pred))

plt.scatter(y_test, y_test_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--')
plt.xlabel("Observed Sale Price")
plt.ylabel("Predicted Sale Price")
plt.title("Simple Linear Regression (GrLivArea)")
plt.show()


## 1.4 Multiple Linear Regression (with Scaling)  

💡 **Tips**  

- **Why multiple regression?**  
  - Instead of using only one feature (like `GrLivArea`), we now include **several predictors** to improve accuracy.  
  - Each feature contributes to predicting `SalePrice` while controlling for the others.  

- **Feature scaling**  
  - `StandardScaler()` standardizes features so they have mean = 0 and standard deviation = 1.  
  - This is important when predictors are measured on very different scales (e.g., `GrLivArea` in square feet vs. `GarageCars` in counts).  
  - Scaling makes training more stable and ensures that features are treated fairly.  

- **Pipeline**  
  - `make_pipeline()` chains preprocessing steps (scaling) and model training into one object.  
  - This ensures scaling is always applied consistently during both training and testing.  

- **Evaluation**  
  - `RMSE` (Root Mean Squared Error): measures average prediction error in the same units as the target (thousands of dollars here).  
  - `R²`: proportion of variance in house prices explained by the model (closer to 1 = better).  
  - Always check both **training** and **test** performance.  

- **Visualization**  
  - Plot **Predicted vs. Observed** prices for the test set.  
  - The red dashed line represents perfect predictions (`y = x`).  
  - A good model has points close to this line, showing accurate predictions.  


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

model2 = make_pipeline(StandardScaler(), LinearRegression())
model2.fit(X_train, y_train)

y_train_pred = model2.predict(X_train)
y_test_pred = model2.predict(X_test)

print("Training RMSE:", mean_squared_error(y_train, y_train_pred, squared=False))
print("Training R²:", r2_score(y_train, y_train_pred))
print("Test RMSE:", mean_squared_error(y_test, y_test_pred, squared=False))
print("Test R²:", r2_score(y_test, y_test_pred))

plt.scatter(y_test, y_test_pred, alpha=0.6)
plt.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], "r--")
plt.xlabel("Observed Sale Price")
plt.ylabel("Predicted Sale Price")
plt.title("Multiple Linear Regression (Scaled Features)")
plt.show()


## 1.5 Polynomial Regression (No Regularization)  
### 1.5.1 Polynomial Regression — Single Feature  
💡 **Tips**  

- **PolynomialFeatures:**  
    - `PolynomialFeatures(d)` expands a feature into higher-order terms.  
    - Example (degree=3):  
      - Input: \(x\)  
      - Output: \([1, x, x^2, x^3]\)  
    - Here we only use **GrLivArea** and expand it into polynomial terms.  

- **Why polynomial regression?**  
    - A simple linear model (straight line) may miss nonlinear patterns.  
    - Polynomial regression allows the model to "bend" and capture curved relationships.  
    
- **Overfitting risk:**  
    - Training error **always decreases** as degree increases (the model fits the training data better).  
    - Test error may start **increasing** at higher degrees → this is overfitting.  

- **Visualization strategy:**  
    - 1. Plot polynomial fits for **degrees 1, 2, 5, and 10**.  
       - Compare how the curve shape changes as degree increases.  
    - 2. Plot **train vs. test error** across degrees (e.g., 1–13).  
       - Use `plt.yscale("log")` since MSE values can differ by orders of magnitude.  
- 3. **Smooth grid for plotting**  
     - Instead of plotting predictions only at training data points, create a **smooth grid** of values for the feature:  
       ```python
       X_plot = np.linspace(X1.min(), X1.max(), 200).reshape(-1, 1)
       ```  
     - `np.linspace(..., 200)` generates **200 evenly spaced values** across the feature’s range.  
     - This makes the prediction curve **smooth and continuous**, which is much clearer than plotting only at raw data points.  
     
- **Loop options:**  
    - **Using index `i`:**  
    
```python
      for i in range(len(degrees_to_plot)):
          d = degrees_to_plot[i]
          plt.subplot(2, 2, i+1)  # subplot index from i is used because subplot indices start at 1, not 0.



In [ ]:
from sklearn.preprocessing import PolynomialFeatures

# Choose a few polynomial degrees to visualize
degrees_to_plot = [1, 2, 5, 10]

# Create a smooth grid for plotting
X_plot = np.linspace(X1.min(), X1.max(), 200).reshape(-1, 1)

plt.figure(figsize=(12, 8))

i = 0
for i in range(len(degrees_to_plot)):
    d = degrees_to_plot[i]
    model = make_pipeline(
        PolynomialFeatures(d, include_bias=True),
        StandardScaler(),
        LinearRegression()
    )
    model.fit(X1_train, y_train)
    y_plot = model.predict(X_plot)

    plt.subplot(2, 2, i+1)
    plt.scatter(X1_train, y_train, color="blue", alpha=0.5, label="Train data")
    plt.plot(X_plot, y_plot, color="red", linewidth=2, label=f"Degree {d}")
    plt.xlabel("GrLivArea")
    plt.ylabel("SalePrice")
    plt.title(f"Polynomial Fit (Degree {d})")
    plt.legend()

plt.tight_layout()
plt.show()

💡 **Tips for Training vs. Test Error Curve**  

**Looping through degrees**  
- `degrees = range(1, 14)` → tests polynomial degrees from 1 (linear) to 13 (very flexible).  
- Inside the loop, for each degree:  
  - Expand features using `PolynomialFeatures(d)`.  
  - Standardize with `StandardScaler()` (important for stability).  
  - Fit a `LinearRegression()` model.  

**Error calculation**  
- Predict on both training and test sets.  
- Compute **Mean Squared Error (MSE)** for train and test predictions.  
- Append results to `train_errors` and `test_errors` lists.  

**Plotting the errors**  
- `plt.plot(degrees, train_errors, label="Train MSE")` → shows how training error changes with model complexity.  
- `plt.plot(degrees, test_errors, label="Test MSE")` → shows generalization error.  
- `plt.yscale("log")` → useful because MSE can vary by orders of magnitude.  

**Interpreting the curves**  
- Training error **always decreases** as degree increases (model fits training data better).  
- Test error decreases at first but then **increases at higher degrees** (overfitting).  
- The gap between train and test error reveals the **bias–variance trade-off**:  
  - Small gap, both errors high → **underfitting**.  
  - Large gap, test error much higher → **overfitting**.  

**Key takeaway**  
- This plot is a diagnostic tool:  
  - Identify the polynomial degree where test error is lowest.  
  - Avoid models with too high a degree (overfit) or too low (underfit).  


In [ ]:
degrees = range(1, 14)
train_errors, test_errors = [], []

for d in degrees:
    model = make_pipeline(
        PolynomialFeatures(d, include_bias=True),
        StandardScaler(),
        LinearRegression()
    )
    model.fit(X1_train, y_train)
    y_train_pred = model.predict(X1_train)
    y_test_pred = model.predict(X1_test)

    train_errors.append(mean_squared_error(y_train, y_train_pred))
    test_errors.append(mean_squared_error(y_test, y_test_pred))

plt.plot(degrees, train_errors, label="Train MSE")
plt.plot(degrees, test_errors, label="Test MSE")
plt.xlabel("Polynomial Degree")
plt.ylabel("MSE")
plt.yscale("log")
plt.title("Polynomial Regression (GrLivArea, No Regularization)")
plt.legend()
plt.show()


### 1.5.2 Polynomial Regression — Multiple Features  

💡 **Tips**  

- **Polynomial expansion with multiple predictors**  
  - `PolynomialFeatures(d)` also works when we have multiple input features.  
  - Example (degree=2, with features $x_1, x_2$):  
    - Generates: $ [1, x_1, x_2, x_1^2, x_2^2, x_1x_2] $  
  - For 3 features (`GrLivArea`, `BedroomAbvGr`, `GarageCars`), expansion adds squared terms and interaction terms.  

- **Feature explosion**  
  - As the degree increases, the number of features grows rapidly.  
  - This can cause **multicollinearity** (redundant features) and overfitting.  

- **Error analysis**  
  - Like with the single feature case, training error decreases with degree.  
  - Test error may increase at higher degrees if the model is too complex.  

- **Visualization**  
  - Plot training vs. test MSE for degrees 1–5.  
  - Use a **logarithmic y-axis** (`plt.yscale("log")`) for better readability across large error differences.  


In [ ]:
degrees = range(1, 6)
train_errors, test_errors = [], []

for d in degrees:
    model = make_pipeline(
        PolynomialFeatures(d, include_bias=True),
        StandardScaler(),
        LinearRegression()
    )
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    train_errors.append(mean_squared_error(y_train, y_train_pred))
    test_errors.append(mean_squared_error(y_test, y_test_pred))

plt.plot(degrees, train_errors, label="Train MSE")
plt.plot(degrees, test_errors, label="Test MSE")
plt.yscale("log")
plt.xlabel("Polynomial Degree")
plt.ylabel("MSE")
plt.title("Polynomial Regression (Three Features, No Regularization)")
plt.legend()
plt.show()


## 1.6 Ridge Regularization  

💡 **Conceptual Overview**  

- **Why Ridge?**  
  - Polynomial regression with high degrees can be very flexible → risk of **overfitting**.  
  - Overfitting symptoms:  
    - Training error → very low.  
    - Test error → high.  

- **Ridge regression (L2 regularization)**  
  - Adds a penalty term to the cost function:  
    $
    J(w) = \frac{1}{2m} \sum_{i=1}^m \big(y^{(i)} - \hat{y}^{(i)}\big)^2 + \alpha \sum_{j=1}^n w_j^2
    $  
  - The penalty discourages large coefficients.  

- **Effect of α (alpha)**  
  - α = 0 → equivalent to ordinary least squares.  
  - Small α → weak penalty, model close to unregularized regression.  
  - Large α → strong penalty, coefficients shrink more, smoother model.  

- **Key points**  
  - The penalty term $\sum w_j^2$ is **quadratic** (a smooth parabola).  
  - During optimization, coefficients are pushed to become smaller.  
  - Ridge keeps all predictors but reduces their influence. 
  - Improves stability and generalization.  

---

### 1.6.1 Ridge Regularization – Train Error Curves  
💡 **Tips**  

**Setup:**  
- Compare **no regularization** vs. Ridge with:  
  - Weak regularization: $\alpha = 0.1$  
  - Strong regularization: $\alpha = 100$  
- Polynomial degrees: from 1 to 5.  
- Errors are tracked separately for training and test sets.  

**Loop for each α value:**  
- Initialize empty lists:  
  - `training_errors_alpha_*` → training MSE for all degrees.  
  - `test_errors_alpha_*` → test MSE for all degrees.  
- For each polynomial degree:  
  1. Expand features with `PolynomialFeatures(d)`.  
  2. Standardize with `StandardScaler()` (important for Ridge).  
  3. Fit `Ridge(alpha=...)` on training data.  
  4. Predict on both training and test sets.  
  5. Compute mean squared error (MSE).  
  6. Append errors to the lists.  

**Plotting strategy:**  
- **Dashed gray line** = baseline (no regularization).  
- **Colored solid lines** = Ridge regression curves (e.g., α=0.1 and α=100 in different colors).  
- Plot training errors vs. polynomial degree for each α separately (or together).  

**Key takeaway:**  
- Ridge **raises training error** compared to no regularization (since coefficients are shrunk).  
- As α increases, the model becomes **simpler and smoother** → fits the training data less perfectly.  
- The trade-off: weaker training fit but **potentially lower test error** (better generalization) at higher degrees.  


In [ ]:
from sklearn.linear_model import Ridge

degrees = range(1, 6)
alpha_weak = 0.1
training_errors_alpha_weak = []
test_errors_alpha_weak = []

for d in degrees:
    model = make_pipeline(
        PolynomialFeatures(d, include_bias=True),
        StandardScaler(),
        Ridge(alpha=alpha_weak)
    )
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    training_errors_alpha_weak.append(mean_squared_error(y_train, y_train_pred))
    test_errors_alpha_weak.append(mean_squared_error(y_test, y_test_pred))

# Plot training error curves
plt.plot(degrees, train_errors, label="Train MSE (No Reg)", linestyle="--", color="gray")
plt.plot(degrees, training_errors_alpha_weak, label=f"Train MSE (Ridge α={alpha_weak})")

plt.xlabel("Polynomial Degree")
plt.ylabel("MSE")
plt.title("Effect of Ridge Regularization on Train Error")
plt.legend()
plt.show()


In [ ]:
degrees = range(1, 6)
alpha_strong = 100
training_errors_alpha_strong = []
test_errors_alpha_strong = []

for d in degrees:
    model = make_pipeline(
        PolynomialFeatures(d, include_bias=True),
        StandardScaler(),
        Ridge(alpha=alpha_strong)
    )
    model.fit(X_train, y_train)
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    training_errors_alpha_strong.append(mean_squared_error(y_train, y_train_pred))
    test_errors_alpha_strong.append(mean_squared_error(y_test, y_test_pred))

# Plot training error curves
plt.plot(degrees, train_errors, label="Train MSE (No Reg)", linestyle="--", color="gray")
plt.plot(degrees, training_errors_alpha_strong, label=f"Train MSE (Ridge α={alpha_strong})")

plt.xlabel("Polynomial Degree")
plt.ylabel("MSE")
plt.title("Effect of Ridge Regularization on Train Error")
plt.legend()
plt.show()


In [ ]:
# Plot training error curves
plt.plot(degrees, train_errors, label="Train MSE (No Reg)", linestyle="--", color="gray")
plt.plot(degrees, training_errors_alpha_weak, label=f"Train MSE (Ridge α={alpha_weak})")
plt.plot(degrees, training_errors_alpha_strong, label=f"Train MSE (Ridge α={alpha_strong})")

plt.xlabel("Polynomial Degree")
plt.ylabel("MSE")
plt.title("Effect of Ridge Regularization on Train Error")
plt.legend()
plt.show()

### 1.6.2 Ridge Regularization – Test Error Curves  
💡 **Tips:**  

**Goal:**  
- Compare how **test error** changes with polynomial degree under three conditions:  
  1. No regularization (baseline).  
  2. Ridge with \(\alpha = 0.1\) (weak penalty).  
  3. Ridge with \(\alpha = 100\) (strong penalty).  

**Plotting:**  
- `plt.plot(degrees, test_errors, ...)` → baseline test error (no regularization), dashed gray line.  
- `plt.plot(degrees, test_errors_alpha_weak, ...)` → Ridge with weak penalty.  
- `plt.plot(degrees, test_errors_alpha_strong, ...)` → Ridge with strong penalty.  
- Each curve shows how **generalization error** changes as model complexity (degree) increases.  

**Interpretation:**  
- **No regularization:**  
  - Test error decreases at first but often **rises sharply at high degrees** (overfitting).  
- **Weak Ridge ($\alpha = 0.1$):**  
  - Test error improves slightly vs. baseline but may still increase at high degrees.  
- **Strong Ridge ($\alpha = 100$):**  
  - Test error curve is **flatter and more stable**, showing Ridge’s ability to reduce variance.  

**Key takeaway:**  
- Ridge may not always minimize test error, but it **stabilizes performance** across degrees.  
- This demonstrates the **bias–variance tradeoff**:  
  - Larger α → higher bias (train error increases).  
  - But variance decreases → test error stabilizes, improving **generalization**.  



In [ ]:
# Plot test error curves
plt.plot(degrees, test_errors, label="Train MSE (No Reg)", linestyle="--", color="gray")
plt.plot(degrees, test_errors_alpha_weak, label=f"Train MSE (Ridge α={alpha_weak})")
plt.plot(degrees, test_errors_alpha_strong, label=f"Train MSE (Ridge α={alpha_strong})")

plt.xlabel("Polynomial Degree")
plt.ylabel("MSE")
plt.title("Effect of Ridge Regularization on Test Error")
plt.legend()
plt.show()

## Further Reading: Lasso Regularization – Conceptual Overview  

**Concept**  

- **Lasso Regression (L1 Regularization)** is another technique to control overfitting.  
- Unlike Ridge (which penalizes squared coefficients), Lasso penalizes the **absolute values** of coefficients.  
- This penalty has a unique property: it can shrink some coefficients **exactly to zero**, effectively removing less important features.  

---

### Lasso Cost Function  

$
J(w) = \frac{1}{2m} \sum_{i=1}^m \Big( y^{(i)} - \hat{y}^{(i)} \Big)^2 + \alpha \sum_{j=1}^n |w_j|
$  

- First term = standard mean squared error.  
- Second term = L1 penalty, proportional to the absolute value of coefficients.  
- **α (alpha)** controls regularization strength:  
  - **α = 0** → equivalent to ordinary least squares.  
  - **Small α** → weak penalty, most coefficients remain nonzero.  
  - **Large α** → strong penalty, many coefficients shrink exactly to zero.  

---

### Key Features of Lasso  

- The L1 penalty introduces **sharp corners at zero** in the optimization landscape.  
- At these corners, the optimizer may set coefficients **exactly to zero**.  
- This means Lasso can **perform feature selection**, keeping only the most important predictors.  
- Like Ridge, Lasso reduces overfitting, but it also creates **sparse models** (simpler, easier to interpret).  

---

### Ridge vs. Lasso  

- **Ridge (L2)** → shrinks coefficients but keeps all features.  
  - “Shrink but keep everything.”  
- **Lasso (L1)** → shrinks and can eliminate features.  
  - “Shrink and throw away what’s not useful.”  
  
---
### Example: Applying Lasso Regression  

```python
from sklearn.linear_model import Lasso

# Polynomial degree for feature expansion
degree = 3  

# Build pipeline: polynomial expansion + scaling + Lasso regression
lasso_model = make_pipeline(
    PolynomialFeatures(degree=degree, include_bias=True),
    StandardScaler(),
    Lasso(alpha=0.1, max_iter=5000)  # alpha controls regularization strength
)



# Part 2. Logistic Regression Tutorial with the Covertype Dataset  

In this part, we will learn how to apply **logistic regression** for classification, using the **Covertype dataset** from scikit-learn.  

Learning Objectives:

- Understand the basics of **logistic regression** for classification problems.  
- Implement **binary logistic regression** using one or more features.  
- Extend logistic regression to handle **multiclass classification**.  
- Use **polynomial feature expansion** to model nonlinear relationships, while recognizing the risk of **overfitting**. 
- Apply **regularization (Ridge / Lasso)** to improve generalization and compare their effects.  
- Evaluate classification models using **accuracy** and **classification reports** (precision, recall, F1).  



## 2.1 Load Dataset  

💡 **Tips**  

- **Dataset source**  
  - Loaded using `fetch_openml("covertype", version=3, as_frame=True)` from the OpenML repository.  
  - Setting `as_frame=True` returns the data as a **pandas DataFrame**, which is convenient for exploration and preprocessing.  
  - Originally created by the **U.S. Forest Service** to classify forest cover types.  

- **Dataset size**  
  - Contains **581,012 rows × 54 features**.  
  - Each row corresponds to a **30×30 meter patch of forest**.  
  - Features include:  
    - **Elevation** (meters)  
    - **Aspect** (azimuthal direction, degrees)  
    - **Slope** (steepness, degrees)  
    - **Horizontal & vertical distances** to hydrology, roadways, and fire points  
    - **Hillshade measures** at 9am, noon, and 3pm  
    - **Wilderness_Area** indicators (binary one-hot encoded, 4 values)  
    - **Soil_Type** indicators (binary one-hot encoded, 40 values)  

- **Target variable**  
  - `Cover_Type`: categorical label with **7 possible forest cover classes**:  
    1. Spruce/Fir  
    2. Lodgepole Pine  
    3. Ponderosa Pine  
    4. Cottonwood/Willow  
    5. Aspen  
    6. Douglas-fir  
    7. Krummholz  

- **Why subsetting?**  
  - Training logistic regression (especially with polynomial features) on all **580k rows** would be too slow for classroom or notebook experiments.  
  - Instead of taking the first chunk of rows, we now **sample 1000 rows from each class without replacement**:  
    - Ensures the dataset is **balanced across all 7 classes**.  
    - Keeps training and visualization **fast** while still reflecting all cover types.  
  - In real-world machine learning, we would use the **full dataset** to train and evaluate models.  

- **Feature and target variables**  
  - `X`: the 54 predictor variables (environmental + categorical).  
  - `y`: the target (`Cover_Type`), stored as integers.  

- **Quick check**  
  - `print("Original shape:", X.shape)` → dataset size before sampling.  
  - `print("Sampled shape:", X.shape)` → confirms the balanced subset size.  
  - `y.value_counts()` → shows class distribution (should be 1000 per class).  
  - `X.head()` → displays the first few rows for inspection.  


In [ ]:
from sklearn.datasets import fetch_openml
import pandas as pd
import warnings
warnings.filterwarnings("ignore", category=UserWarning)
warnings.filterwarnings("ignore", category=RuntimeWarning)

cover = fetch_openml(name="covertype", version=3, as_frame=True)

X_cover = cover.data
y_cover = cover.target.astype(int)
del cover # delete the large dataset to release memory

print("Original shape:", X_cover.shape)

In [ ]:
import pandas as pd

def sample_classes(X, y, classes, n_per_class=1000, random_state=42):
    """
    Sample equal number of rows without replacement from specified classes.

    Parameters
    ----------
    X : pd.DataFrame
        Feature matrix.
    y : pd.Series
        Target vector.
    classes : list
        List of class labels to sample (e.g., [3, 4]).
    n_per_class : int, default=1000
        Number of samples per class.
    random_state : int, default=42
        Random seed for reproducibility.

    Returns
    -------
    X_sampled : pd.DataFrame
        Subsampled features.
    y_sampled : pd.Series
        Subsampled targets.
    """
    sampled_X, sampled_y = [], []
    for cls in classes:
        mask = (y == cls)
        X_cls = X[mask]
        y_cls = y[mask]
        X_sub = X_cls.sample(n=n_per_class, replace=False, random_state=random_state)
        y_sub = y_cls.loc[X_sub.index]
        sampled_X.append(X_sub)
        sampled_y.append(y_sub)
    return pd.concat(sampled_X), pd.concat(sampled_y)

In [ ]:
# Example: sample 1000 rows from class 3 and 1000 from class 4
X, y = sample_classes(X_cover, y_cover, classes=[1, 2, 3, 4, 5, 6, 7], n_per_class=1000)

print("Sampled shape:", X.shape)
print("Class counts:\n", y.value_counts())


## 2.2 Select and Visualize Features  

💡 **Tips**  

- **Feature types**  
  - The Covertype dataset includes two kinds of features:  
    - **Continuous (numeric):** e.g., `Elevation`, `Slope`, `Aspect`.  
    - **Categorical indicators (0/1 flags):** e.g., `Wilderness_Area*`, `Soil_Type*`.  
  - In this step, we focus on continuous features, since they are easier to visualize.  

- **Feature distributions by class:**  
    - `plt.boxplot()` shows how a feature’s values are distributed across cover types.  
    - Example: comparing **Elevation** across cover types may reveal which forests occur at higher vs. lower altitudes.  
- Boxplots are helpful for spotting both **differences** and **overlaps** among classes.  

- **Subplots setup:**  
    - `plt.subplot(131)`, `plt.subplot(132)`, `plt.subplot(133)` create a row of 3 side-by-side plots.  
    - Each subplot displays one feature (`Elevation`, `Aspect`, or `Slope`).  
    - `plt.tight_layout()` automatically adjusts spacing to avoid overlapping titles and labels.  

- **Takeaway**  
  - These plots provide a **first look** at how the target classes differ in terms of continuous features.  
  - Later, logistic regression will try to exploit these differences to separate classes.  



In [ ]:
import matplotlib.pyplot as plt
import numpy as np

features_to_plot = ["Elevation", "Aspect", "Slope"]

plt.figure(figsize=(18, 5))

# Subplot 1: Elevation
plt.subplot(131)
data = [X[y == cls]["Elevation"] for cls in np.unique(y)]
plt.boxplot(data, labels=np.unique(y))
plt.title("Elevation vs Cover Type")
plt.xlabel("Cover Type")
plt.ylabel("Elevation")

# Subplot 2: Aspect
plt.subplot(132)
data = [X[y == cls]["Aspect"] for cls in np.unique(y)]
plt.boxplot(data, labels=np.unique(y))
plt.title("Aspect vs Cover Type")
plt.xlabel("Cover Type")
plt.ylabel("Aspect")

# Subplot 3: Slope
plt.subplot(133)
data = [X[y == cls]["Slope"] for cls in np.unique(y)]
plt.boxplot(data, labels=np.unique(y))
plt.title("Slope vs Cover Type")
plt.xlabel("Cover Type")
plt.ylabel("Slope")

plt.tight_layout()
plt.show()


### 2.3 Binary Logistic Regression (One Feature)  

💡 **Tips**  

- **Why binary classification?**  
  - The Covertype dataset has 7 classes, but logistic regression is easiest to understand in the **binary case**.  
  - Here, we focus only on two classes (e.g., class 3 vs. class 6) to demonstrate the basics.  

- **Feature selection**  
  - We use only **one feature**: `Elevation`.  
  - This makes it simple to visualize and interpret how logistic regression separates two classes.  

- **Data splitting**  
  - `train_test_split()` splits the data into training (70%) and test (30%) sets.  
  - Training set → used to fit the model.  
  - Test set → used to evaluate generalization performance.  

- **Model training**  
  - `LogisticRegression(max_iter=2000, n_jobs=-1)`  
    - Creates the logistic regression model.  
    - `max_iter=2000`: allows more iterations so the solver converges reliably.  
    - `n_jobs=-1`: uses all CPU cores for faster computation.  
  - **Penalty behavior**:  
    - By default, scikit-learn applies **L2 penalty** (Ridge-style regularization).  
    - You can override with `penalty="none"` for no regularization (pure logistic regression).  
    - With certain solvers (`liblinear`, `saga`), you can also use:  
      - `penalty="l1"` → Lasso-style regularization.  
      - `penalty="elasticnet"` → mix of L1 and L2.  
  - `.fit(X_train, y_train)`  
    - Fits the model to the training data.  
    - Learns the best parameters (weights for the feature and a bias term).  
    - Unlike linear regression, logistic regression outputs **probabilities**, which are then thresholded (default 0.5) to assign class labels.  

- **Evaluation metrics**  
  - `accuracy_score`: overall proportion of correctly classified samples.  
  - `classification_report`: shows **precision, recall, and F1-score** for each class.  
    - **Precision** = out of predicted positives, how many are correct.  
    - **Recall** = out of actual positives, how many are detected.  
    - **F1-score** = harmonic mean of precision and recall (balances both).  

- **Key takeaway**  
  - Using just one feature, the model is very simple, but it clearly demonstrates how logistic regression works.  
  - Later, we will extend to multiple features and multiclass classification for better performance.  


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# Subset: classes 3 and 6 only
mask = y.isin([3, 6])
X_bin, y_bin = X[mask], y[mask]

# One feature
X_small = X_bin[["Elevation"]]

X_train, X_test, y_train, y_test = train_test_split(X_small, y_bin, test_size=0.3, random_state=42)

logreg = LogisticRegression(max_iter=2000, n_jobs=-1, penalty = "none")
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


### Interpreting Classification Report  

**Class 3:**  
- **Precision = 0.53** → Of all samples predicted as class 3, 53% were correct.  
- **Recall = 0.45** → Out of the 302 true class 3 samples, only 45% were correctly identified.  
- **F1 = 0.49** → The balance of precision and recall; performance is modest.  
- **Support = 302** → Number of class 3 samples in the test set.  

**Class 6:**  
- **Precision = 0.52** → Of all samples predicted as class 6, 52% were correct.  
- **Recall = 0.60** → Out of the 298 true class 6 samples, 60% were correctly identified.  
- **F1 = 0.56** → Better than class 3, mainly due to higher recall.  
- **Support = 298** → Number of class 6 samples in the test set.  

**Overall accuracy = 0.52**  
- The model correctly classified about **52%** of test samples — only slightly above random guessing (50% for two balanced classes).  

**Macro avg (equal weight for each class):**  
- Precision = 0.52, Recall = 0.52, F1 = 0.52.  
- Shows the model performs almost the same across both classes.  

**Weighted avg (accounts for class sizes):**  
- Nearly identical to macro average, since classes are roughly balanced.  

**Key Takeaways**
- With just **one feature (Elevation)**, logistic regression performs **only slightly better than chance 50%**.  
- The model struggles to separate classes 3 and 6, as their elevation ranges overlap heavily.  
- Class 6 is predicted with slightly better recall, while class 3 is harder to identify correctly.  
- This example highlights the **limitations of single-feature models** — adding more features will likely improve performance.  


## 2.4 Binary Logistic Regression (Three Features)  

💡 **Tips**  

- **Why extend to multiple features?**  
  - With only one feature (`Elevation`), the model can only separate classes along a single axis.  
  - Adding **more features** (`Slope` and `Aspect`) gives logistic regression more information to distinguish between classes.  
  - This allows the model to create more complex **linear decision boundaries** in higher dimensions.  

- **Feature selection**  
  - We now use three continuous features:  
    - `Elevation`  
    - `Slope`  
    - `Aspect`  

- **Data splitting**  
  - The same procedure as before: `train_test_split()` splits into training and test sets.  
  - Ensures that model performance is evaluated on unseen data.  

- **Model training**  
  - We reuse the same logistic regression model (`logreg.fit()`).  
  - Logistic regression automatically extends to multiple features by learning a **weight** for each feature plus an intercept.  
  - The decision boundary is now a **plane** in 3D space (instead of a line in 2D).  

- **Evaluation metrics**  
  - `accuracy_score`: percentage of correct predictions.  
  - `classification_report`: provides **precision, recall, and F1-score** for each class.  
  - Comparison to the one-feature model shows whether the extra features improve classification.  

- **Key takeaway**  
  - Logistic regression scales naturally from one to multiple features.  
  - By including more features, we typically improve accuracy, as the model can better capture the differences between classes.  



In [ ]:
X_small = X_bin[["Elevation", "Slope", "Aspect"]]

X_train, X_test, y_train, y_test = train_test_split(X_small, y_bin, test_size=0.3, random_state=42)

logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


### Interpreting Classification Report  

**Class 3:**  
- **Precision = 0.54** → Of all samples predicted as class 3, 54% were correct.  
- **Recall = 0.52** → Out of the 302 true class 3 samples, 52% were correctly identified.  
- **F1 = 0.53** → Balanced precision/recall performance, but modest overall.  
- **Support = 302** → Number of class 3 samples in the test set.  

**Class 6:**  
- **Precision = 0.53** → Of all samples predicted as class 6, 53% were correct.  
- **Recall = 0.55** → Out of the 298 true class 6 samples, 55% were correctly identified.  
- **F1 = 0.54** → Slightly better balance than class 3 due to higher recall.  
- **Support = 298** → Number of class 6 samples in the test set.  

**Overall accuracy = 0.54**  
- The model correctly classified about **54%** of test samples.  
- This is only **slightly above random guessing (50%)**, so the separation between classes remains weak.  

**Macro avg (equal weight per class):**  
- Precision = 0.54, Recall = 0.54, F1 = 0.54.  
- Indicates both classes perform about the same.  

**Weighted avg (accounts for class sizes):**  
- Nearly identical to macro avg, since classes are balanced (302 vs. 298).  

**Key Takeaways**  
- Adding **Slope** and **Aspect** did not significantly improve performance compared to using only Elevation (≈52%).  
- The logistic regression model struggles because **classes 3 and 6 overlap heavily across these continuous features**.  
- Both classes are predicted with similar (but weak) quality → no major bias toward one class.  
- This shows the **limits of linear logistic regression with few features**. To improve accuracy, the model may need **additional features** or **more flexible, higher-degree representations**.


## 2.5 Polynomial Features (No Regularization)  

💡 **Tips**  

- **Why polynomial features?**  
  - `PolynomialFeatures(degree=d)` expands the input features by adding:  
    - **Squared terms** (e.g., `Elevation²`, `Slope²`).  
    - **Interaction terms** (e.g., `Slope × Aspect`).  
  - This allows logistic regression (normally a **linear classifier**) to model **nonlinear decision boundaries**.  

- **Flexibility vs. overfitting**  
  - Increasing the polynomial degree makes the model more flexible.  
  - **Training accuracy**: always improves (or stays the same).  
  - **Test accuracy**: may decrease at higher degrees because the model starts fitting noise → **overfitting**.  

- **Pipeline setup**  
  - `PolynomialFeatures(degree=d)`: generates polynomial and interaction terms.  
  - `StandardScaler()`: standardizes features to mean = 0 and standard deviation = 1 (improves numerical stability).  
  - `LogisticRegression(...)`: fits the model.  
    - `max_iter=2000`: allows more optimization iterations to ensure convergence.  
    - `solver="lbfgs"`: quasi-Newton optimization method, efficient for medium- to large-sized datasets.  
    - **Penalty behavior:**  
      - By default, scikit-learn applies **L2 penalty** (Ridge-style regularization).  
      - You can override with `penalty="none"` for no regularization (pure logistic regression).  
      - With certain solvers (`liblinear`, `saga`), you can also use `penalty="l1"` (Lasso) or `penalty="elasticnet"`.  

- **Other solver options**  
  - `"liblinear"` → good for small datasets; supports L1 and L2.  
  - `"saga"` → scalable for large datasets; supports L1, L2, and elastic net.  
  - `"newton-cg"`, `"sag"`, `"lbfgs"` → efficient for large-scale L2-regularized models.  

- **Why no learning rate parameter?**  
  - Unlike SGD-based training, these solvers automatically choose step sizes (learning rates) internally using **line search or adaptive strategies**.  
  - That’s why `LogisticRegression` does **not** expose a `learning_rate` parameter.  
  - If you want explicit learning rate control, use `SGDClassifier(loss="log_loss")`.  

- **Looping over degrees**  
  - Train the model for polynomial degrees 1–15.  
  - Record both training and test accuracy.  

- **Visualization**  
  - `plt.plot(degrees, train_accs, ...)`: plots training accuracy curve.  
  - `plt.plot(degrees, test_accs, ...)`: plots test accuracy curve.  
  - Comparing the two curves reveals:  
    - **Underfitting** → both train/test accuracy low (small degree).  
    - **Overfitting** → train accuracy high, test accuracy lower (large degree).  

- **Key takeaway**  
  - Polynomial features greatly increase model complexity.  
  - Without regularization (`penalty="none"`), high-degree polynomials tend to overfit.  
  - Logistic regression **by default includes L2 regularization** — you must explicitly set `penalty="none"` to turn it off.  
  - Always compare training and test curves to spot the bias–variance tradeoff.  


In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler

degrees = range(1, 16)
train_accs, test_accs = [], []

for d in degrees:
    poly_model = make_pipeline(
        PolynomialFeatures(degree=d, include_bias=True),
        StandardScaler(),
        LogisticRegression(max_iter=2000, solver="lbfgs", penalty="none", n_jobs=-1)
    )
    poly_model.fit(X_train, y_train)
    train_accs.append(accuracy_score(y_train, poly_model.predict(X_train)))
    test_accs.append(accuracy_score(y_test, poly_model.predict(X_test)))
    print(f"degree={d}")

plt.plot(degrees, train_accs, marker="o", label="Train Accuracy")
plt.plot(degrees, test_accs, marker="s", label="Test Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Accuracy")
plt.title("Accuracy vs Polynomial Degree (No Regularization)")
plt.legend()
plt.grid(True)
plt.show()


## 2.6 Regularization (L2 Penalty)  
💡 **Tips**  

- **Why regularization?**  
    - Logistic regression with **polynomial features** can become too flexible and overfit the training data.  
    - **L2 penalty (Ridge-style regularization)** helps control complexity by shrinking coefficients.  
    - This reduces variance and makes the model generalize better.  

- **The `C` parameter**  
    - `C` is the **inverse of regularization strength**:  
      - Small `C` (e.g., 0.1) → **strong regularization** → simpler model, higher bias, lower variance.  
      - Large `C` (e.g., 100) → **weak regularization** → more flexible model, lower bias, higher variance (risk of overfitting).  
    - `penalty="none"` → baseline with **no regularization**.  

- **Experiment setup**  
    - Test polynomial degrees **1–15**.  
    - For each degree, fit logistic regression models with:  
      - `C=100` (weak regularization).  
      - `C=0.1` (strong regularization).  
      - `penalty="none"` (no regularization).  
    - Record **training accuracy** and **test accuracy** for comparison.  

- **Code structure**  
    - `train_accs_*`, `test_accs_*` → lists that store accuracies for each condition.  
    - **Loop** over degrees → fit models with different regularization.  
    - Save results → used later for plotting.  

- **Plotting**  
    - **Left subplot** → Training accuracy vs. polynomial degree.  
      - Shows how well models fit training data.  
    - **Right subplot** → Test accuracy vs. polynomial degree.  
      - Shows how well models generalize to unseen data.  

- **Key takeaway**  
    - No regularization → high training accuracy, but test accuracy drops (overfitting).  
    - Strong regularization → lower training accuracy, but test accuracy is **more stable** (better generalization).  
    - Regularization balances the **bias–variance trade-off**.  


In [ ]:
degrees = range(1, 16)
C_strong = 100
train_accs_C_strong = []
test_accs_C_strong = []

for d in degrees:
    model = make_pipeline(
        PolynomialFeatures(degree=d, include_bias=True),
        StandardScaler(),
        LogisticRegression(max_iter=2000, solver="lbfgs", C=C_strong, penalty="l2", n_jobs=-1)
    )
    model.fit(X_train, y_train)
    train_accs_C_strong.append(accuracy_score(y_train, model.predict(X_train)))
    test_accs_C_strong.append(accuracy_score(y_test, model.predict(X_test)))
    print(f"degree={d}")

# Training accuracy
plt.figure(figsize=(12, 4))

plt.subplot(121)

plt.plot(degrees, train_accs_C_strong, marker="o", label=f"C={C_strong}, small regularization")
plt.plot(degrees, train_accs, marker="o", label="Train Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Training Accuracy")
plt.title("Training Accuracy vs Degree\n (With and Without Regularization)")
plt.legend()
plt.grid(True)

# Test accuracy
plt.subplot(122)

plt.plot(degrees, test_accs_C_strong, marker="s", label=f"C={C_strong}, small regularization")
plt.plot(degrees, test_accs, marker="s", label="Test Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy vs Degree\n (With and Without Regularization)")
plt.legend()
plt.grid(True)


In [ ]:
degrees = range(1, 16)
C_weak = 0.1
train_accs_C_weak = []
test_accs_C_weak = []

for d in degrees:
    model = make_pipeline(
        PolynomialFeatures(degree=d, include_bias=True),
        StandardScaler(),
        LogisticRegression(max_iter=2000, solver="lbfgs", C=C_weak, penalty="l2")
    )
    model.fit(X_train, y_train)
    train_accs_C_weak.append(accuracy_score(y_train, model.predict(X_train)))
    test_accs_C_weak.append(accuracy_score(y_test, model.predict(X_test)))
    print(f"degree={d}")

# Training accuracy
plt.figure(figsize=(12, 4))

plt.subplot(121)

plt.plot(degrees, train_accs_C_weak, marker="o", label=f"C={C_weak}, large regularization")
plt.plot(degrees, train_accs, marker="o", label="Train Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Training Accuracy")
plt.title("Training Accuracy vs Degree\n (With and Without Regularization)")
plt.legend()
plt.grid(True)

# Test accuracy
plt.subplot(122)

plt.plot(degrees, test_accs_C_weak, marker="s", label=f"C={C_weak}, large regularization")
plt.plot(degrees, test_accs, marker="s", label="Test Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy vs Degree\n (With and Without Regularization)")
plt.legend()
plt.grid(True)


In [ ]:
# Training accuracy
plt.figure(figsize=(12, 4))

plt.subplot(121)

plt.plot(degrees, train_accs_C_strong, marker="o", label=f"C={C_strong}, small regularization")
plt.plot(degrees, train_accs_C_weak, marker="o", label=f"C={C_weak}, large regularization")
plt.plot(degrees, train_accs, marker="o", label="Train Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Training Accuracy")
plt.title("Training Accuracy vs Degree\n (With and Without Regularization)")
plt.legend()
plt.grid(True)

# Test accuracy
plt.subplot(122)

plt.plot(degrees, test_accs_C_strong, marker="s", label=f"C={C_strong}, small regularization")
plt.plot(degrees, test_accs_C_weak, marker="s", label=f"C={C_weak}, large regularization")
plt.plot(degrees, test_accs, marker="s", label="Test Accuracy")
plt.xlabel("Polynomial Degree")
plt.ylabel("Test Accuracy")
plt.title("Test Accuracy vs Degree\n (With and Without Regularization)")
plt.legend()
plt.grid(True)


## 2.7 Extend to Three Classes  

💡 **Tips**  

- **From binary to multiclass**  
  - Logistic regression naturally supports **multiclass classification**.  
  - By setting `multi_class="multinomial"`, the model uses the [**softmax function**](https://www.geeksforgeeks.org/deep-learning/what-is-softmax-classifier/) to assign probabilities across all classes.  
  - This is more appropriate than training multiple one-vs-rest models when classes are not just binary.  

- **Dataset selection**  
  - Instead of just two classes (e.g., 3 vs. 6), we now include **three classes** (3, 5, and 6).  
  - This allows us to see how logistic regression handles more than two categories at once.  

- **Model setup**  
  - `LogisticRegression(max_iter=2000, multi_class="multinomial", solver="lbfgs")`:  
    - `multi_class="multinomial"` → enables softmax-based multiclass classification.  
    - `solver="lbfgs"` → efficient optimization method that supports multinomial logistic regression.  
    - `max_iter=2000` → ensures the model has enough iterations to converge.  
    **By default**  
        - `penalty="l2"` → Ridge-style regularization.  
        - `C=1.0` → inverse of regularization strength.  
          - Smaller `C` = stronger penalty (more shrinkage).  
          - Larger `C` = weaker penalty (closer to no regularization).  

- **Evaluation metrics**  
  - `accuracy_score`: overall proportion of correctly classified samples.  
  - `classification_report`: provides **precision, recall, and F1-score** for each of the three classes.  
  - These metrics help evaluate if the model is biased toward certain classes or performs evenly across them.  

- **Key takeaway**  
  - Logistic regression extends naturally to handle multiple classes.  
  - By using the multinomial setting, the model can learn decision boundaries that separate all classes simultaneously.  
  - Performance depends on how distinct the feature distributions are for each class.  


In [ ]:
mask = y.isin([3, 5, 6])
X_multi, y_multi = X[mask], y[mask]

X_train, X_test, y_train, y_test = train_test_split(X_multi, y_multi, test_size=0.3, random_state=42)

logreg_multi = LogisticRegression(max_iter=2000, multi_class="multinomial", solver="lbfgs")
logreg_multi.fit(X_train, y_train)

y_pred = logreg_multi.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


# Assignment 2 

- Begin with a **new Jupyter Notebook** containing only your **code, figures, and explanations**.  
- Save and submit your notebook with the following filename format:  

  **Lastname_Firstname_NetID_Assignment2.ipynb**  


# Part 1: Linear Regression with Ames Housing Dataset

---

## **1. Multiple Linear Regression (≥ 2 features)**  (1 pt)
- Split the dataset into **training (70%)** and **test (30%)** sets.  
- Start with one key feature (e.g., `GrLivArea`) and add three others (e.g., `GarageCars`, `LotArea`, `TotRmsAbvGrd`).  
- Use a pipeline with `StandardScaler()` and `LinearRegression()`.  
- Fit the model and report:  
  - Training RMSE and R²  
  - Test RMSE and R²  
- Plot **Predicted vs Observed** for the test set.  
- Compare your results with earlier sections (1.3 Simple Linear Regression and 1.4 Multiple Regression).  

---

## **2. Polynomial Regression (no regularization)**  (1 pt)
- Fit polynomial regression models with degrees **1 through 10** using the selected features.  
- Record **training and test MSE** at each degree.  
- Plot both training error vs degree and test error vs degree side by side
- Discuss: At which polynomial degree does **overfitting** begin to appear?  

---

## **3. Ridge Regularization**  (1 pt)
- Repeat polynomial regression, but now add Ridge regularization.  
- Train models with **α = 0.1, α = 1, and α = 100**.  
- Plot training error vs degree and test error vs degree, comparing with the no-regularization baseline.  
- Discuss how Ridge regularization helps reduce **overfitting** in high-degree polynomial models.  

---


# Part 2: Logistic Regression with Covertype Dataset

---

## **1. Load Data and Visualization** (1 pt)
- Select **two classes other than 3 vs 6** (e.g., 3 vs 4, or 2 vs 5).  
- Choose one numerical feature (e.g., `Elevation`).  
- Plot histograms for this feature, showing both classes in the same figure.  
- Fit a `LogisticRegression` model using this single feature.  
- Report:  
  - Training accuracy and test accuracy  
  - Classification report (precision, recall, F1-score)  

---

## **2. Logistic Regression with Multiple Features** (1 pt)
- Select **three or more numerical features** (e.g., `Elevation`, `Slope`, `Aspect`).  
- Fit a `LogisticRegression` model.  
- Report:  
  - Training accuracy and test accuracy  
  - Classification report  
- Compare the performance with Problem 1.  

---

## **3. Polynomial Features and Ridge Regularization** (1 pt)
- Choose a relatively **high polynomial degree** (e.g., ≥ 5).  
- Select two **C values** (e.g., 0.01, 0.1, 1, 10, or 100) — remember:  
  - **Small C → strong regularization**  
  - **Large C → weak regularization**  
- Fit three logistic regression models and compare:  
  - **No penalty** (`penalty="none"`)  
  - **L2 penalty (Ridge)** (`penalty="l2"`, default)  
- Report:  
  - Training accuracy and test accuracy  
  - Classification report (precision, recall, F1-score)  

---

## **4. Bonus: Lasso Regularization** (Bonus 1 pts)  

- Choose a relatively **high polynomial degree** (e.g., ≥ 5).  
- Select several **C values** (e.g., 0.01, 0.1, 1, or 10) — remember:  
  - **Small C → strong regularization**  
  - **Large C → weak regularization**  
- Fit three logistic regression models and compare:  
  - **No penalty** (`penalty="none"`)  
  - **L1 penalty (Lasso)** (`penalty="l1"`, requires solver `"liblinear"` or `"saga"`)  
- Report **training and test accuracy** for each case.  
- Discuss:  
  - How are your results different from Probelm 3?  
  - How does the choice of C affect the results?  


💡 **Tips:**  
- `LogisticRegression` uses **L2 penalty (Ridge)** by default.  
- To use **L1 penalty (Lasso)**, set `penalty="l1"` and `solver="liblinear"` or `solver="saga"` (since not all solvers support L1).  
- To remove regularization, set `penalty="none"` (available with `solver="lbfgs"` or `solver="saga"`).  


# Part 3: Flux Tower Data Regression (4 pts)

Apply what you have learned to build regression models on **daily Flux Tower data (2005–2021) in Assignment 1**.  

---

## **Instructions**
1. Split the dataset:  
   - Train on **2005–2020**  
   - Predict **2021**  

2. Start simple:  
   - Start with the **single feature that you think is most related** with photosynthesis.  
   - Fit a simple linear regression model using this feature.  

3. Add complexity:  
   - Gradually include more predictors (e.g., temperature, radiation, vapor pressure deficit (VPD), soil water content, precipitation).  
   - Explore polynomial regression with degrees ≥ 2.  

4. Prevent overfitting:  
   - When using higher-order polynomial models, also test **Ridge** or **Lasso** regularization.  
   - Compare performance with and without regularization.  

5. Model selection:  
   - Choose your **final model**.  
   - Provide a rationale: Why this model? Consider the **bias–variance tradeoff**, **accuracy**, and **interpretability**.  

---

## **Deliverables**
- Report **RMSE** and **R²** for both the training and test sets.  
- Provide a plot of **Predicted vs. True Photosynthesis** for the year 2021.  
- Submit your **code** along with a concise written explanation (≤ 200 words) that:  
  - Summarizes your exploration process.  
  - Justifies your final model choice.  

